In [19]:
import chipwhisperer as cw
import numpy as np

import simon_64_128
import helper

from helper import KeyHypothesis
from measurement import Measurements

# Import Trace

In [26]:
project = cw.import_project("./traces/demo_simon_plain_2000/trace.zip", overwrite=True)
# project = cw.import_project("./traces/demo_simon_masked_2000/trace.zip", overwrite=True)

In [27]:
TOTAL_NUM_MEASUREMENTS = len(project.traces)
TOTAL_VALS_PER_MEASUREMENT = len(project.traces[0].wave)

NUM_MEASUREMENTS = 1000
VALS_PER_MEASUREMENT = 1000

# Transform Trace Project into Measurement object

In [28]:
%matplotlib widget

plaintexts = np.zeros((TOTAL_NUM_MEASUREMENTS, 2), dtype=np.uint32)
ciphertexts = np.zeros((TOTAL_NUM_MEASUREMENTS, 2), dtype=np.uint32)
powers = np.zeros((TOTAL_NUM_MEASUREMENTS, TOTAL_VALS_PER_MEASUREMENT), dtype=np.uint32)

for i, trace in enumerate(project.traces):

    pt = trace.textin
    ct = trace.textout

    plaintexts[i,0] = pt[0] << 24 | pt[1] << 16 | pt[2] << 8 | pt[3]
    plaintexts[i,1] = pt[4] << 24 | pt[5] << 16 | pt[6] << 8 | pt[7]

    ciphertexts[i,0] = ct[0] << 24 | ct[1] << 16 | ct[2] << 8 | ct[3]
    ciphertexts[i,1] = ct[4] << 24 | ct[5] << 16 | ct[6] << 8 | ct[7]
    powers[i,:] = trace.wave

all_measurements = Measurements(plaintexts, ciphertexts, powers)
measurements = Measurements(plaintexts[0:NUM_MEASUREMENTS], ciphertexts[0:NUM_MEASUREMENTS], powers[0:NUM_MEASUREMENTS, 0:VALS_PER_MEASUREMENT])


In [29]:
key = np.array([0x05168c8d, 0x2f811f8b, 0xcfda9184, 0xd6d57d44], dtype=np.uint32) # measurement demo_simon_plain_2000
# key = np.array([0xcaa395ba, 0xcb3a7aad, 0xbd28a2f0, 0x47d8993e], dtype=np.uint32) # measurement demo_simon_masked_2000

pt = measurements[0].plaintext
ct = measurements[0].ciphertext

ct_expect, _ = simon_64_128.encrypt_block(pt, key)

print(f"Plaintext:           {hex(pt[0])}, {hex(pt[1])}")
print(f"Ciphertext:          {hex(ct[0])}, {hex(ct[1])}")
print(f"Expected Ciphertext: {hex(ct_expect[0])}, {hex(ct_expect[1])}")

Plaintext:           0xc0b0c7c4, 0x4aed4906
Ciphertext:          0x3360376c, 0x3a5c30ea
Expected Ciphertext: 0x3360376c, 0x3a5c30ea


In [30]:
print("Full Measurement:")
print(f"- Plaintext shape:  {all_measurements.plaintext.shape}")
print(f"- Ciphertext shape: {all_measurements.ciphertext.shape}")
print(f"- Power shape:      {all_measurements.power.shape}")

print("Reduced Measurement:")
print(f"- Plaintext shape:  {measurements.plaintext.shape}")
print(f"- Ciphertext shape: {measurements.ciphertext.shape}")
print(f"- Power shape:      {measurements.power.shape}")

Full Measurement:
- Plaintext shape:  (2000, 2)
- Ciphertext shape: (2000, 2)
- Power shape:      (2000, 5000)
Reduced Measurement:
- Plaintext shape:  (1000, 2)
- Ciphertext shape: (1000, 2)
- Power shape:      (1000, 1000)


# Attacking X after Add Round Key

In [31]:
ATTACKED_STATE = "ADD_ROUND_KEY"
GUESSED_BITS_PER_STEP = 6
CORR_DIFF_THRESHOLD = 0.02

# start with empty key as hypothesis as root for the search tree
empty_key = np.zeros((4,), dtype=np.uint32)
empty_mask = np.zeros((4,), dtype=np.uint32)
start_hypo = KeyHypothesis(empty_key, empty_mask)

# variables to maintain during attack
promising_hypos = [start_hypo]
guessed_bits_mask = np.zeros((4,), dtype=np.uint32)

for attacked_round in range(4):
    print(f"Start attacking round {attacked_round}.")
    num_guessed_bits = 0
    while True:
        if guessed_bits_mask[3- attacked_round] == 0xFFFFFFFF:
            print(f"Finished attacking round {attacked_round}. {len(promising_hypos)} promising hypotheses left.")
            break
        else:
            # Add newly guessed bits to the mask.
            for i in range(num_guessed_bits, num_guessed_bits + GUESSED_BITS_PER_STEP):
                guessed_bits_mask[3 - attacked_round] |= (1 << i)
            num_guessed_bits += GUESSED_BITS_PER_STEP
        print(f"- Perform CPA on {helper.bits_count(guessed_bits_mask[3 - attacked_round])} guessed bits.")

        # Generate sub hypotheses for all previously identified promising hypotheses
        sub_hypos: list[KeyHypothesis] = []
        for h in promising_hypos:
            sub_hypos.extend(h.get_sub_hypos(guessed_bits_mask))

        print(f"- Find correlations for {len(sub_hypos)} hypotheses.")
        helper.calc_corrs_for_hypos(sub_hypos, measurements, attacked_round, ATTACKED_STATE)
        promising_hypos = helper.filter_hypos(sub_hypos, CORR_DIFF_THRESHOLD)

        print(f"- Found {len(promising_hypos)} promising hypotheses:")
        for hypo in promising_hypos:
            print(f"  - {helper.array_to_hex_str(hypo.key)}: correlation: {hypo.corr}")

key_found = False
if len(promising_hypos) > 0:
    # Do final check against ciphertext
    for hypo in promising_hypos:
        ciphertext, _ = simon_64_128.encrypt_block(measurements[0].plaintext, hypo.key)
        if np.array_equal(measurements[0].ciphertext, ciphertext):
            extracted_key = hypo.key
            key_found = True
            print(f"Extracted Key: {helper.array_to_hex_str(extracted_key)}")
            break
    
if not key_found:
    print("Could not determine key")

Start attacking round 0.
- Perform CPA on 6 guessed bits.
- Find correlations for 64 hypotheses.
- Found 2 promising hypotheses:
  - 0x00000000 0x00000000 0x00000000 0x00000004: correlation: -0.47345666222373584
  - 0x00000000 0x00000000 0x00000000 0x0000003B: correlation: 0.47345666222373584
- Perform CPA on 12 guessed bits.
- Find correlations for 128 hypotheses.
- Found 2 promising hypotheses:
  - 0x00000000 0x00000000 0x00000000 0x00000D44: correlation: -0.5759164981971923
  - 0x00000000 0x00000000 0x00000000 0x000002BB: correlation: 0.575916498197192
- Perform CPA on 18 guessed bits.
- Find correlations for 128 hypotheses.
- Found 2 promising hypotheses:
  - 0x00000000 0x00000000 0x00000000 0x00017D44: correlation: -0.684637429523913
  - 0x00000000 0x00000000 0x00000000 0x000282BB: correlation: 0.6846374295239129
- Perform CPA on 24 guessed bits.
- Find correlations for 128 hypotheses.
- Found 2 promising hypotheses:
  - 0x00000000 0x00000000 0x00000000 0x00D57D44: correlation: -0

# Attacking X after AND-Gate

In [32]:
ATTACKED_STATE = "AND_GATE"
GUESSED_BITS_PER_STEP = 6
CORR_DIFF_THRESHOLD = 0.03

# start with empty key as hypothesis as root for the search tree
empty_key = np.zeros((4,), dtype=np.uint32)
empty_mask = np.zeros((4,), dtype=np.uint32)
start_hypo = KeyHypothesis(empty_key, empty_mask)

# variables to maintain during attack
promising_hypos = [start_hypo]
guessed_bits_mask = np.zeros((4,), dtype=np.uint32)

for attacked_round in range(4):
    print(f"Start attacking round {attacked_round}.")
    num_guessed_bits = 0
    while True:
        if guessed_bits_mask[3- attacked_round] == 0xFFFFFFFF:
            print(f"Finished attacking round {attacked_round}. {len(promising_hypos)} promising hypotheses left.")
            break
        else:
            # Add newly guessed bits to the mask. Place the bits in distance of 7 to each other
            for i in range(num_guessed_bits, num_guessed_bits + GUESSED_BITS_PER_STEP):
                bit_pos = i*7 % 32
                guessed_bits_mask[3 - attacked_round] |= (1 << bit_pos)
            num_guessed_bits += GUESSED_BITS_PER_STEP

        print(f"- Perform CPA on {helper.bits_count(guessed_bits_mask[3 - attacked_round])} guessed bits.")

        # Generate sub hypotheses for all previously identified promising hypotheses
        sub_hypos: list[KeyHypothesis] = []
        for h in promising_hypos:
            sub_hypos.extend(h.get_sub_hypos(guessed_bits_mask))

        print(f"- Find correlations for {len(sub_hypos)} hypotheses.")
        helper.calc_corrs_for_hypos(sub_hypos, measurements, attacked_round, ATTACKED_STATE)
        promising_hypos = helper.filter_hypos(sub_hypos, CORR_DIFF_THRESHOLD)

        print(f"- Found {len(promising_hypos)} promising hypotheses:")
        for hypo in promising_hypos:
            print(f"  - {helper.array_to_hex_str(hypo.key)}: correlation: {hypo.corr}")

key_found = False
if len(promising_hypos) > 0:
    # Do final check against ciphertext
    for hypo in promising_hypos:
        ciphertext, _ = simon_64_128.encrypt_block(measurements[0].plaintext, hypo.key)
        if np.array_equal(measurements[0].ciphertext, ciphertext):
            extracted_key = hypo.key
            key_found = True
            print(f"Extracted Key: {helper.array_to_hex_str(extracted_key)}")
            break
    
if not key_found:
    print("Could not determine key")

Start attacking round 0.
- Perform CPA on 6 guessed bits.
- Find correlations for 64 hypotheses.
- Found 1 promising hypotheses:
  - 0x00000000 0x00000000 0x00000000 0x10004000: correlation: -0.43261048928825857
- Perform CPA on 12 guessed bits.
- Find correlations for 64 hypotheses.
- Found 1 promising hypotheses:
  - 0x00000000 0x00000000 0x00000000 0x90006440: correlation: -0.5847891936684961
- Perform CPA on 18 guessed bits.
- Find correlations for 64 hypotheses.
- Found 2 promising hypotheses:
  - 0x00000000 0x00000000 0x00000000 0x90116444: correlation: -0.6726614265903897
  - 0x00000000 0x00000000 0x00000000 0x90916444: correlation: -0.6827576075222919
- Perform CPA on 24 guessed bits.
- Find correlations for 128 hypotheses.
- Found 4 promising hypotheses:
  - 0x00000000 0x00000000 0x00000000 0xD0117444: correlation: -0.7201064705333604
  - 0x00000000 0x00000000 0x00000000 0xD4117444: correlation: -0.7203095328901188
  - 0x00000000 0x00000000 0x00000000 0xD0917444: correlation: 